In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn.init as init
from torchsummary import summary

In [103]:
class GradReverse(torch.autograd.Function):
    """
    Extension of grad reverse layer
    """
    @staticmethod
    def forward(ctx, x, constant):
        ctx.constant = constant
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        grad_output = grad_output.neg() * ctx.constant
        return grad_output, None

    def grad_reverse(x, constant):
        return GradReverse.apply(x, constant)

In [24]:
class Network(nn.Module):

    def __init__(self,kernel_size=3):
        super(Network, self).__init__()
        self.extractor = Extractor()
        self.classifier = Class_classifier(num_class=2)
        self.domain = Domain_classifier(domain_class=5)
                
    def forward(self, x, hp_lambda=0):
        print(x.shape)
        x = self.extractor(x)
        print(x.shape)
        cls = self.classifier(x)
        print(x.shape)
        dom = self.domain(x,hp_lambda)
        print(x.shape)
        
        return cls,dom

In [25]:
help(summary)

Help on function summary in module torchsummary.torchsummary:

summary(model, input_size, batch_size=-1, device='cuda')



In [27]:
model = Network()
model.cuda()
summary(model,(1,246,13))

torch.Size([2, 1, 246, 13])
torch.Size([2, 1, 246, 13])
torch.Size([2, 2880])
torch.Size([2, 2880])
torch.Size([2, 2880])
domain torch.Size([2, 2880])
torch.Size([2, 2880])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 244, 12]             112
       BatchNorm2d-2          [-1, 16, 244, 12]              32
            Conv2d-3          [-1, 32, 242, 10]           4,640
       BatchNorm2d-4          [-1, 32, 242, 10]              64
         Dropout2d-5          [-1, 32, 242, 10]               0
            Conv2d-6          [-1, 32, 244, 12]           9,248
       BatchNorm2d-7          [-1, 32, 244, 12]              64
         Dropout2d-8          [-1, 32, 244, 12]               0
            Conv2d-9           [-1, 64, 122, 6]          18,496
      BatchNorm2d-10           [-1, 64, 122, 6]             128
        Dropout2d-11           [-1, 64, 122, 6]           

/home/mhealthra2/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/home/mhealthra2/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [17]:
class Extractor(nn.Module):

    def __init__(self):
        super(Extractor, self).__init__()
        self.conv0 = nn.Conv2d(1, 16, kernel_size=(3,2),stride=1)   ## change with input shape
        self.bn0 = nn.BatchNorm2d(16)
        
        # Res block 1
        self.conv1 = nn.Conv2d(16, 32, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv11 = nn.Conv2d(32, 32, kernel_size=(3,3),stride=(1,1),padding=(2,2))
        self.bn11 = nn.BatchNorm2d(32)
        
        # Res block 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3,3), stride=(1,1),padding=(1,1))
        self.bn2 = nn.BatchNorm2d(64)
        self.conv21 = nn.Conv2d(64, 64, kernel_size=(3,3), stride=(2,2),padding=(1,1))
        self.bn21 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 64, kernel_size=(5,3), stride=(2,1),padding=(1,1)) ### change with input shape
        self.bn3 = nn.BatchNorm2d(64)
        
        self.drop = nn.Dropout2d(0.5)
        
        
    def forward(self, x):
        print(x.shape)
        x = F.relu(self.bn0(self.conv0(x)))
        
        #Res block 1
        x1 = self.drop(F.relu(self.bn1(self.conv1(x))))
        x1 = F.relu(F.max_pool2d(self.drop(self.bn11(self.conv11(x1))), 2))
        x = torch.cat((x,torch.zeros_like(x)), axis=1)
        x = F.max_pool2d(x,2)
        x = x+x1
        
        #Res block 2
        x1 = self.drop(F.relu(self.bn2(self.conv2(x))))
        x1 = F.relu(self.drop(self.bn21(self.conv21(x1))))
        x = torch.cat((x,torch.zeros_like(x)), axis=1)
        x = F.max_pool2d(x,2)        
        x = x+x1
        
        #last conv
        x = self.drop(F.relu(self.bn3(self.conv3(x))))
        x = F.max_pool2d(x,(2,1))  ### change withinput
        x = x.view(-1, 64*3*15)
        print(x.shape)
        
        return x

In [18]:
net = Extractor()
net.cuda()
summary(net,(1,246,13))

torch.Size([2, 1, 246, 13])
torch.Size([2, 2880])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 244, 12]             112
       BatchNorm2d-2          [-1, 16, 244, 12]              32
            Conv2d-3          [-1, 32, 242, 10]           4,640
       BatchNorm2d-4          [-1, 32, 242, 10]              64
         Dropout2d-5          [-1, 32, 242, 10]               0
            Conv2d-6          [-1, 32, 244, 12]           9,248
       BatchNorm2d-7          [-1, 32, 244, 12]              64
         Dropout2d-8          [-1, 32, 244, 12]               0
            Conv2d-9           [-1, 64, 122, 6]          18,496
      BatchNorm2d-10           [-1, 64, 122, 6]             128
        Dropout2d-11           [-1, 64, 122, 6]               0
           Conv2d-12            [-1, 64, 61, 3]          36,928
      BatchNorm2d-13            [-1, 64, 61, 3]      

In [ ]:
len(list(net.parameters()))

In [ ]:
64*3*15*2

In [113]:
class Class_classifier(nn.Module):

    def __init__(self, num_class,in_feature=64*3*15,intermediate_nodes=100):
        super(Class_classifier, self).__init__()
        # self.fc1 = nn.Linear(50 * 4 * 4, 100)
        # self.bn1 = nn.BatchNorm1d(100)
        # self.fc2 = nn.Linear(100, 100)
        # self.bn2 = nn.BatchNorm1d(100)
        # self.fc3 = nn.Linear(100, 10)
        self.fc1 = nn.Linear(in_feature, intermediate_nodes)
        self.fc2 = nn.Linear(intermediate_nodes, num_class)
        
        self.relu = nn.ReLU()
        self.soft = nn.Softmax(dim=1)
        
    def forward(self, x):
        # logits = F.relu(self.bn1(self.fc1(input)))
        # logits = self.fc2(F.dropout(logits))
        # logits = F.relu(self.bn2(logits))
        # logits = self.fc3(logits)
        logits = self.relu(self.fc1(x))
        logits = self.fc2(F.dropout(logits))
        logits = self.soft(logits)

        return logits

In [114]:
class Domain_classifier(nn.Module):

    def __init__(self,domain_class,in_feature=64*3*15,intermediate_nodes=100):
        super(Domain_classifier, self).__init__()
        # self.fc1 = nn.Linear(50 * 4 * 4, 100)
        # self.bn1 = nn.BatchNorm1d(100)
        # self.fc2 = nn.Linear(100, 2)
        
        self.fc1 = nn.Linear(in_feature, intermediate_nodes)
        self.fc2 = nn.Linear(intermediate_nodes, domain_class)
        
        self.relu = nn.ReLU()
        self.soft = nn.Softmax(dim=1)
    def forward(self, x, constant):
        x = GradReverse.grad_reverse(x, constant)
        # logits = F.relu(self.bn1(self.fc1(input)))
        # logits = F.log_softmax(self.fc2(logits), 1)
        print('domain',x.shape)
        logits = self.relu(self.fc1(x))
        logits = self.soft(self.fc2(logits))

        return logits

In [107]:
class Branch(nn.Module):

    def __init__(self,c_in, c_out, kernel_size=5,stride=1,dropout = 0.5):
        super(Branch, self).__init__()
        self.conv1 = nn.Conv1d(c_in, c_out, kernel_size=kernel_size,stride=stride)
        self.bn1 = nn.BatchNorm1d(c_out)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(dropout)
        self.conv2 = nn.Conv1d(c_out, 2*c_out, kernel_size=kernel_size,stride=stride)
        self.bn2 = nn.BatchNorm1d(c_out*2)
    def forward(self, x):
        x = self.drop(self.relu(self.bn1(self.conv1(x))))
        x = self.drop(self.relu(self.bn2(self.conv2(x))))
        return x

In [108]:
class Res_block(nn.Module):
    def __init__(self,c_in,c_out,kernel_size=5,stride=1,dropout=0.5,padding=2):
        super(Res_block,self).__init__()
        self.conv1 = nn.Conv1d(c_in,c_out,kernel_size=kernel_size,stride=stride,padding=padding)
        self.bn1 = nn.BatchNorm1d(c_out)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(dropout)
        self.conv2 = nn.Conv1d(c_out, c_out, kernel_size=kernel_size,stride=1,padding=padding)
        self.bn2 = nn.BatchNorm1d(c_out)
        self.pool = nn.MaxPool1d(stride)
        
        
    def forward(self,x):
        x1 = self.drop(self.relu(self.bn1(self.conv1(x))))
        x1 = self.drop(self.relu(self.bn2(self.conv2(x1))))
        x = self.pool(x)
        x = torch.cat((x,torch.zeros_like(x)), axis=1)
        x = x+x1
        return x

In [109]:
class Smallnet(nn.Module):

    def __init__(self,num_class,domain_class):
        super(Smallnet, self).__init__()
        self.branch1 = Branch(1,8)
        self.res_block1 = Res_block(16,32,stride=2)
        self.res_block2 = Res_block(32,64,stride=2)
        self.pool = nn.MaxPool1d(2)
        self.conv = nn.Conv1d(64,128,kernel_size=5,stride=2)
        self.drop = nn.Dropout(0.5)
        
        self.classifier = Class_classifier(num_class=num_class,in_feature=9856,intermediate_nodes=50)
        self.domain = Domain_classifier(domain_class=domain_class,in_feature=9856,intermediate_nodes=80)
    def forward(self, x, hp_lambda=0):
        print(x.shape)
        x = self.branch1(x)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.drop(self.pool(self.conv(self.pool(x))))
        x = x.view(-1,9856)
        print(x.shape)
        cls = self.classifier(x)
        dom = self.domain(x,hp_lambda)
        return cls,dom
    

In [115]:
n = Smallnet(2,5)

In [116]:
summary(n.cuda(),(1,2500))

torch.Size([2, 1, 2500])
torch.Size([2, 9856])
domain torch.Size([2, 9856])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 8, 2496]              48
       BatchNorm1d-2              [-1, 8, 2496]              16
              ReLU-3              [-1, 8, 2496]               0
           Dropout-4              [-1, 8, 2496]               0
            Conv1d-5             [-1, 16, 2492]             656
       BatchNorm1d-6             [-1, 16, 2492]              32
              ReLU-7             [-1, 16, 2492]               0
           Dropout-8             [-1, 16, 2492]               0
            Branch-9             [-1, 16, 2492]               0
           Conv1d-10             [-1, 32, 1246]           2,592
      BatchNorm1d-11             [-1, 32, 1246]              64
             ReLU-12             [-1, 32, 1246]               0
          Dropout-13       